Plants Diseases Classification Model :

In [4]:
# Used Libraries & Independences :
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [3]:
# Our Dataset Properities :
image_size = 256
images_batch = 32

In [ ]:
# Dataset Ingestion :
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ""
)